In [ ]:
# Import necessary functions from your script
# This assumes your notebook is in a location where it can import from 'cli.llm_article_generator'
# Adjust the import path if your notebook is in a different directory relative to the 'cli' folder.
# For example, if your notebook is in 'english-language-helper/notebooks/',
# you might need to adjust sys.path or use relative imports carefully.

import sys
import os

# Add the project root to sys.path to allow imports from cli
project_root = os.path.abspath(os.path.join(os.getcwd(), '..')) # Adjust if notebook is not in 'notebooks' subdir
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Now try to import from the cli module
try:
    from cli.llm_article_generator import get_firestore_client, generate_article
    import firebase_admin # To check initialization
    from firebase_admin import firestore # For server_timestamp
except ImportError as e:
    print(f"Error importing from llm_article_generator: {e}")
    print("Ensure the notebook is in the 'english-language-helper/notebooks' directory,")
    print("or adjust the 'project_root' path accordingly.")
    print(f"Current working directory: {os.getcwd()}")
    print(f"Project root (attempted): {project_root}")
    print(f"Sys.path: {sys.path}")
    # You might need to install dependencies if the error is about missing modules:
    # %pip install firebase-admin click
    raise

# --- Configuration ---
# OPTION 1: Use environment variable (ensure FIREBASE_SERVICE_ACCOUNT_KEY_PATH is set)
# key_file_path = None # Will use environment variable

# OPTION 2: Specify the path to your Firebase service account key JSON file directly
# Replace with the actual path to your key file if not using env var
key_file_path = "G:/FIRESTOREKEYS/english-language-art-helper-firebase-adminsdk-fbsvc-e5e8256254.json" # <--- !!! REPLACE THIS IF NOT USING ENV VAR !!!
# or key_file_path = os.environ.get('FIREBASE_SERVICE_ACCOUNT_KEY_PATH')

In [ ]:
# Article details
article_topic = "The Benefits of Regular Exercise"
article_level_order = 1 # Corresponds to the 'order' field in your 'levels' collection (e.g., GSAT is order 1)

# --- 1. Initialize Firestore Client (reusing the script's function) ---
print(f"Attempting to initialize Firestore with key: {key_file_path or 'from ENV VAR'}...")
db_client = get_firestore_client(key_path=key_file_path)

if db_client:
    print("Firestore client initialized successfully in the notebook.")
    
    # --- 2. (Simulate) LLM Content Generation ---
    # In a real scenario, you'd call your LLM here.
    # We'll use placeholder logic similar to the script.
    print(f"\nSimulating LLM generation for topic: '{article_topic}' at level order: {article_level_order}")

    level_id_for_article = None
    level_name_for_prompt = "Unknown Level"
    try:
        levels_query = db_client.collection('levels').where('order', '==', article_level_order).limit(1)
        level_docs = list(levels_query.stream())
        if level_docs:
            level_id_for_article = level_docs[0].id
            level_name_for_prompt = level_docs[0].get('nameEnglish', "Unknown Level")
            print(f"Found level: '{level_name_for_prompt}' (ID: {level_id_for_article}) for order {article_level_order}.")
        else:
            print(f"ERROR: No level found with order {article_level_order}. Cannot generate article.")
            # exit() # Or handle error appropriately in notebook
    except Exception as e:
        print(f"Error querying levels collection: {e}")
        # exit()

    if level_id_for_article:
        llm_title = f"The Amazing Advantages of {article_topic}"
        llm_content = (
            f"This article explores {article_topic}, focusing on its importance for learners at the {level_name_for_prompt} level. "
            f"Regular exercise offers numerous physical and mental benefits that contribute to a healthier and happier life.\n\n"
            f"**Physical Benefits:** Discuss improved cardiovascular health, weight management, and stronger muscles/bones.\n"
            f"**Mental Benefits:** Cover stress reduction, better mood, and enhanced cognitive function.\n\n"
            f"Engaging in physical activity consistently can significantly improve overall well-being."
        )
        llm_tags = [tag.strip().lower() for tag in article_topic.split(',')] + ["health", "fitness", "well-being", "llm-generated-notebook", level_id_for_article]
        llm_tags = list(set(filter(None, llm_tags)))


        # --- 3. Prepare Article Data for Firestore ---
        article_data_to_publish = {
            'title': llm_title,
            'content': llm_content,
            'levelIds': [level_id_for_article],
            'tags': llm_tags,
            'createdAt': firestore.SERVER_TIMESTAMP, # Use SERVER_TIMESTAMP
            'updatedAt': firestore.SERVER_TIMESTAMP,
            'scrapedAt': firestore.SERVER_TIMESTAMP, # For LLM, this is like generation timestamp
            'sourceUrl': f"llm_notebook_generated/topic_{article_topic.lower().replace(' ', '_')}",
            'hasComprehensionQuestions': False, # Initially false
        }
        print("\nPrepared article data:")
        for key, value in article_data_to_publish.items():
            if key in ['createdAt', 'updatedAt', 'scrapedAt']:
                print(f"  {key}: Server Timestamp")
            else:
                print(f"  {key}: {value}")

        # --- 4. Add to Firestore ---
        try:
            article_collection_ref = db_client.collection('articles')
            # Using add() will auto-generate an ID
            timestamp, new_article_ref = article_collection_ref.add(article_data_to_publish)
            print(f"\nSuccessfully added article to Firestore with ID: {new_article_ref.id}")
            print(f"Timestamp of addition: {timestamp}")
            
            # You can then construct a URL to view it in Firebase Console (replace YOUR_PROJECT_ID)
            # print(f"View in Firestore: https://console.firebase.google.com/project/YOUR_PROJECT_ID/firestore/data/~2Farticles~2F{new_article_ref.id}")

        except Exception as e:
            print(f"\nError adding article to Firestore: {e}")
    else:
        print(f"Could not proceed with article generation as levelId for order {article_level_order} was not found.")
        
else:
    print("Firestore client initialization failed. Please check the key path and Firebase setup.")

# To clean up the Firebase app instance if you re-run this cell multiple times (optional)
# if firebase_admin._apps:
#     firebase_admin.delete_app(firebase_admin.get_app())
#     print("Cleaned up Firebase app instance for re-initialization.")